<a href="https://colab.research.google.com/github/jimmy-io/Insight_project/blob/master/Reddit_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Insight/Project/Reddit\ images

/content/drive/My Drive/Insight/Project/Reddit images


In [1]:
!pip install praw

In [2]:
##Upload reddit credentials so sensitive info is not visible 

from google.colab import files
src = list(files.upload().values())[0]
open('redcreds.py','wb').write(src)
import redcreds

Saving redcreds.py to redcreds.py


In [0]:
import praw 
reddit = praw.Reddit(client_id = redcreds.my_client_id,client_secret=redcreds.my_client_secret, password=redcreds.password, user_agent=redcreds.user_agent, username=redcreds.username)

In [4]:
import requests, re, urllib
import pandas as pd

counter=0
hot_posts = reddit.subreddit('malefashion').top('all', limit = None)

post_list1=[]
counter=0
for post in hot_posts:
  if post.link_flair_text == 'WIWT':
    
    url = (post.url)
    
    list1=[]

    #print(post.url)
    list1.append(post.url)
    #print(post.upvote_ratio)
    list1.append(post.upvote_ratio)
    #print(post.score)
    list1.append(post.score)
    #print(post.id)
    list1.append(post.id)

    counter+=1
    print('This is post #'+ str(counter))
    post_list1.append(list1)

    
post_list1=pd.DataFrame(post_list1)
post_list1


    
    

This is post #1
This is post #2
This is post #3
This is post #4
This is post #5
This is post #6


KeyboardInterrupt: ignored

In [0]:
## image extraction stuff 

filename=re.sub('[^A-Za-z0-9]+', '', post.title)

    filename=str(str(counter)+'+'+filename+'+'+str(post.upvote_ratio))

    counter+=1

    urllib.request.urlretrieve(url, filename+'.jpg')


In [0]:
### Pushshift.io stuff 

import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

In [0]:
def give_me_intervals(start_at, number_of_days_per_interval = 3):
    
    end_at = math.ceil(datetime.utcnow().timestamp())
        
    ## 1 day = 86400,
    period = (86400 * number_of_days_per_interval)

    end = start_at + period
    yield (int(start_at), int(end))

    padding = 1
    while end <= end_at:
        start_at = end + padding
        end = (start_at - padding) + period
        yield int(start_at), int(end)

In [0]:

def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)

    current_tries = 1
    while current_tries < max_retries:
        try:
            response = fire_away(uri)
            return response
        except:
            time.sleep(.150)
            current_tries += 1

    return fire_away(uri)
 

In [0]:
def map_posts(posts):
  return list(map(lambda post: {'id': post['id'],'created_utc': post['created_utc'],'prefix': 't4_'}, posts))

In [0]:
def pull_posts_for(subreddit, start_at, end_at):

    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts( \
        make_request( \
            URI_TEMPLATE.format(subreddit, start_at, end_at, SIZE))['data'])

    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - 10
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format(subreddit, new_start_at, end_at, SIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)

    return post_collections

In [23]:
subreddit = 'Malefashion'

end_at = math.ceil(datetime.utcnow().timestamp())
start_at = math.floor((datetime.utcnow() - timedelta(days=10)).timestamp())

posts = pull_posts_for(subreddit, start_at, end_at)

print('found:', len(posts))
print('unique:', len(np.unique([ post['id'] for post in posts ])))

found: 204
unique: 204


In [16]:
## assert no dups,
sorted_by_occurence = sorted([ (k, len(list(g))) for k,g in itertools.groupby(posts, lambda x: x['id']) ], key=lambda x: x[1], reverse=True)
sorted_by_occurence[:10]

[('eqcip6', 1),
 ('eqcv43', 1),
 ('eqczpt', 1),
 ('eqev04', 1),
 ('eqh777', 1),
 ('eqhijv', 1),
 ('eqhtrq', 1),
 ('eqhwt0', 1),
 ('eqitux', 1),
 ('eqixyc', 1)]

In [17]:
## quick look at the posts,
posts[:10]

[{'created_utc': 1579325965, 'id': 'eqcip6', 'prefix': 't4_'},
 {'created_utc': 1579328267, 'id': 'eqcv43', 'prefix': 't4_'},
 {'created_utc': 1579329134, 'id': 'eqczpt', 'prefix': 't4_'},
 {'created_utc': 1579343420, 'id': 'eqev04', 'prefix': 't4_'},
 {'created_utc': 1579358814, 'id': 'eqh777', 'prefix': 't4_'},
 {'created_utc': 1579360442, 'id': 'eqhijv', 'prefix': 't4_'},
 {'created_utc': 1579362024, 'id': 'eqhtrq', 'prefix': 't4_'},
 {'created_utc': 1579362442, 'id': 'eqhwt0', 'prefix': 't4_'},
 {'created_utc': 1579366523, 'id': 'eqitux', 'prefix': 't4_'},
 {'created_utc': 1579366906, 'id': 'eqixyc', 'prefix': 't4_'}]

In [19]:
posts_from_reddit = []
comments_from_reddit = []

TIMEOUT_AFTER_COMMENT_IN_SECS = .250

for submission_id in np.unique([ post['id'] for post in posts ]):
    submission = reddit.submission(id=submission_id)

    posts_from_reddit.append(submission)

    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comments_from_reddit.append(comment)
        
        if TIMEOUT_AFTER_COMMENT_IN_SECS > 0:
            time.sleep(TIMEOUT_AFTER_COMMENT_IN_SECS)

KeyboardInterrupt: ignored

In [13]:

len(posts_from_reddit)

15